In [2]:
using TensorKit
import TensorKit.×
include("./iPEPS_fSU2/iPEPS.jl")
include("./CTMRG_fSU2/CTMRG.jl")

@show pkgversion(TensorKit)

pkgversion(TensorKit) = v"0.12.1"


v"0.12.1"

In [ ]:
# pspace = GradedSpace{fSU₂}((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
pspace = GradedSpace{fSU₂}((0 => 2), (1//2 => 1))
aspacel = GradedSpace{fSU₂}((0 => 1), (1//2 => 1))
aspacet = GradedSpace{fSU₂}((0 => 1), (1//2 => 2))
aspacer = GradedSpace{fSU₂}((0 => 2), (1//2 => 2))
aspaceb = GradedSpace{fSU₂}((0 => 1), (1//2 => 1), (1 => 1))
# aspacel = aspacer = aspacet = aspaceb = pspace
@show pspace
nothing

In [ ]:
a = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
@show space(a)  # [l, t, p; r, b]
@show space(a') # [r, b; l, t, p]

@tensor tmp[l1, l2, t1, t2; r1, r2, b1, b2] := a[l1, t1, p, r1, b1] * a'[r2, b2, l2, t2, p]
nothing

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [ ]:
A = id(pspace)
@show A
exp!(A)
@show A + A
nothing

In [ ]:
ipeps = iPEPSΓΛ(pspace, aspacel, aspacet, 2, 2; dtype=ComplexF64)

gateNN = TensorMap(randn, ComplexF64, pspace ⊗ pspace, pspace ⊗ pspace)

nothing

In [5]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
aspace = Rep[ℤ₂×SU₂]((1, 1 // 2) => 1)
Fdag = TensorMap(zeros, pspace, pspace ⊗ aspace)
block(Fdag, Irrep[ℤ₂×SU₂](1, 1 // 2))[1, 1] = 1
block(Fdag, Irrep[ℤ₂×SU₂](0, 0))[2, 1] = sqrt(2)
# @show Fdag
@show blocks(Fdag)
@show block(Fdag, Irrep[ℤ₂×SU₂](1, 1 // 2))
@show block(Fdag, Irrep[ℤ₂×SU₂](0, 0))
@show size(block(Fdag, Irrep[ℤ₂×SU₂](0, 0)))
nothing

blocks(Fdag) = TensorKit.SortedVectorDict{TensorKit.ProductSector{Tuple{Z2Irrep, SU2Irrep}}, Matrix{Float64}}((0, 0) => [0.0; 1.4142135623730951;;], (1, 1/2) => [1.0 0.0])
block(Fdag, (Irrep[ℤ₂ × SU₂])(1, 1 // 2)) = [1.0 0.0]
block(Fdag, (Irrep[ℤ₂ × SU₂])(0, 0)) = [0.0; 1.4142135623730951;;]


size(block(Fdag, (Irrep[ℤ₂ × SU₂])(0, 0))) = (2, 1)


In [39]:
pspace = GradedSpace{fSU₂}((0 => 2), (1//2 => 1))
aspace = GradedSpace{fSU₂}((1//2 => 1))
Fdag = TensorMap(zeros, pspace, pspace ⊗ aspace)
for (key, val) in blocks(Fdag)
    if key == Irrep[SU₂](0) ⊠ FermionParity(0)
        val[2, 1] = sqrt(2)
    elseif key == Irrep[SU₂](1/2) ⊠ FermionParity(1)
        val[1, 1] = one(val[1, 1])
    else
        nothing
    end
end
@show blocks(Fdag)
# block(Fdag, GradedSpace{fSU₂}(1 // 2))[1, 1] = 1
# block(Fdag, GradedSpace{fSU₂}(0))[2, 1] = sqrt(2)
# @show Fdag
# @show block(Fdag, Irrep[ℤ₂×SU₂](1, 1 // 2))
# @show block(Fdag, Irrep[ℤ₂×SU₂](0, 0))
# @show size(block(Fdag, Irrep[ℤ₂×SU₂](0, 0)))
nothing

blocks(Fdag) = TensorKit.SortedVectorDict{TensorKit.ProductSector{Tuple{SU2Irrep, FermionParity}}, Matrix{Float64}}((0, 0) => [0.0; 1.4142135623730951;;], (1/2, 1) => [1.0 0.0])
